In [6]:
#necessary libraries
import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt


In [8]:
data=pd.read_csv(os.path.join("data","June2023.csv"))
data.head()

,_id,DATE_TIME,LATITUDE,LONGITUDE,GEOHASH,MAXIMUM_SPEED,MINIMUM_SPEED,AVERAGE_SPEED,NUMBER_OF_VEHICLES
0,1,2023-06-01 00:00:00,28.218384,41.130066,sxk1fp,135,34,67,35
1,2,2023-06-01 00:00:00,29.020386,41.069641,sxk9sk,86,6,48,89
2,3,2023-06-01 00:00:00,28.745728,40.981750,sxk3nh,125,9,60,107
3,4,2023-06-01 00:00:00,28.778687,41.042175,sxk3qz,89,6,38,30
4,5,2023-06-01 00:00:00,28.822632,40.959778,sxk3pb,42,7,19,15


In [ ]:
#Flow (vehicles/hour)

In [ ]:
#İstanbul ilçeleri lontitude-lantitude'ları alınarak değerlere maplenmesi lazım (enlem-boylam yazmayacak da İlçe diye bir sütun olacak)
#datetime - ilçe(feature olarak verebiliriz veya sadece time series yapabiliriz. Her türlü görselleştirme için iyi olur) - Number_of_vehicles olacak
#number_of vehicles için genel bir aralık belirlenmesi lazım (bunu genel değerlere göre inisiyatif alarak yaparız)
## low = 
## medium =
## high =
#ilçelere göre ayırabiliriz ama gerek yok bence İstanbul genelinde bir tahminleme yapıyor oluruz
#Ama biz şöyle yaparsak efektif olur gibi = her saat için avg her ilçedeki num of vehicles
###  __id_______datetime_____ilçe(modellenirken çıkarılır görselleştirme için kullanılır)_____avg_num_of_vehicles
###   

In [ ]:
### 24 saat 1 aydan 24.31 = 744 satırımız olmalı 1 aylık veri için (678 diyor neden çözemedim ama bakarız)
### saatlere göre gruplayıp avg(num_of_vehicles) alınmalı 
##

In [ ]:
# Load your dataset
# Assuming you have a CSV file with columns 'datetime' and 'num_of_vehicles'
#data['datetime'] = pd.to_datetime(data['datetime'])
#data = data.set_index('datetime')

# Ensure the data is sorted chronologically
#data = data.sort_index()

# Plot the original data to visualize the traffic flow
plt.figure(figsize=(15, 6))
plt.plot(data.index, data['num_of_vehicles'])
plt.title('Hourly Traffic Flow Data')
plt.xlabel('Datetime')
plt.ylabel('Number of Vehicles')
plt.show()


In [ ]:
# Normalize the data
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)


In [ ]:
# Create sequences and targets
def create_sequences(data, seq_length):
    sequences = []
    target = []
    
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length]
        sequences.append(seq)
        target.append(label)
    
    return np.array(sequences), np.array(target)

# Define sequence length (e.g., past 24 hours)
sequence_length = 24

# Create sequences and targets
X, y = create_sequences(data_normalized, sequence_length)


In [ ]:
# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(sequence_length, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on Test Set: {loss}')

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test)

In [ ]:
# Inverse transform the predictions to the original scale
predictions_original_scale = scaler.inverse_transform(predictions.reshape(-1, 1))
y_test_original_scale = scaler.inverse_transform(y_test.reshape(-1, 1))

In [ ]:
# Plot the predictions against the actual values
plt.figure(figsize=(15, 6))
plt.plot(data.index[train_size + sequence_length:], y_test_original_scale, label='Actual Traffic Flow')
plt.plot(data.index[train_size + sequence_length:], predictions_original_scale, label='Predicted Traffic Flow')
plt.title('Traffic Flow Prediction with LSTM')
plt.xlabel('Datetime')
plt.ylabel('Number of Vehicles')
plt.legend()
plt.show()

In [ ]:
### extra output that gives 24 hour daily prediction for the day we want to see :
# Select the most recent 24 hours for prediction
latest_data = data.iloc[-sequence_length:]

# Normalize the latest data
latest_data_normalized = scaler.transform(latest_data)

# Create a single sequence for prediction
X_latest = np.array([latest_data_normalized])

# Make predictions
prediction_latest_normalized = model.predict(X_latest)

# Inverse transform the prediction to the original scale
prediction_latest_original_scale = scaler.inverse_transform(prediction_latest_normalized.reshape(-1, 1))

# Print the predicted number of vehicles in the latest 24 hours
print(f'Predicted Number of Vehicles in the Latest 24 Hours: {int(prediction_latest_original_scale[0, 0])}')